In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import sys
import csv
import functools
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re
import string
import statistics

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
#from sklearn import svm
#from sklearn.svm import SVC
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [2]:
udf = pd.read_pickle("./4_COMB_FILE_UNDERSAMPLING")
print(str(udf.shape))
odf = pd.read_pickle("./4_COMB_FILE_OVERSAMPLING")
print(str(odf.shape))
df = pd.read_pickle("./4_COMB_FILE_UNBALANCED.pkl")
print(str(df.shape))

(7528, 49)
(31622, 49)
(25671, 49)


In [3]:
# create bag of words for df
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(df.CLEAN_FOI_TEXT)
df_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
df_dtm.index = df.index
print(df_dtm.shape)

# train-test split for df
X_df = df_dtm
y_df = df["RECALL_CLASS"]

X_df_train, X_df_test, y_df_train, y_df_test = train_test_split(X_df, y_df, test_size = 0.33, random_state = 0)
print(X_df_train.shape)
print(X_df_test.shape)
print(y_df_train.shape)
print(y_df_test.shape)

(25671, 6205)
(17199, 6205)
(8472, 6205)
(17199,)
(8472,)


In [4]:
# create bag of words for udf
cv_u = CountVectorizer(stop_words='english')
data_cv_u = cv_u.fit_transform(udf.CLEAN_FOI_TEXT)
udf_dtm = pd.DataFrame(data_cv_u.toarray(), columns=cv_u.get_feature_names())
udf_dtm.index = udf.index
print(udf_dtm.shape)

# train-test split for udf
X_u = udf_dtm
y_u = udf["RECALL_CLASS"]

X_u_train, X_u_test, y_u_train, y_u_test = train_test_split(X_u, y_u, test_size = 0.33, random_state = 0)
print(X_u_train.shape)
print(X_u_test.shape)
print(y_u_train.shape)
print(y_u_test.shape)

(7528, 3745)
(5043, 3745)
(2485, 3745)
(5043,)
(2485,)


In [5]:
# create bag of words for odf
cv_o = CountVectorizer(stop_words='english')
data_cv_o = cv_o.fit_transform(odf.CLEAN_FOI_TEXT)
odf_dtm = pd.DataFrame(data_cv_o.toarray(), columns=cv_o.get_feature_names())
odf_dtm.index = odf.index
print(odf_dtm.shape)

# train-test split for odf
X_o = odf_dtm
y_o = odf["RECALL_CLASS"]

X_o_train, X_o_test, y_o_train, y_o_test = train_test_split(X_o, y_o, test_size = 0.33, random_state = 0)
print(X_o_train.shape)
print(X_o_test.shape)
print(y_o_train.shape)
print(y_o_test.shape)

(31622, 6159)
(21186, 6159)
(10436, 6159)
(21186,)
(10436,)


In [6]:
# running naive-bayes classifiers

In [7]:
# for unbalanced class data
classifierdf = GaussianNB()
classifierdf.fit(X_df_train, y_df_train)
ydfPredNB = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredNB))
print('\n')
print(classification_report(y_df_test,ydfPredNB))

Unbalanced classes
[[3080  193]
 [2880 2319]]


              precision    recall  f1-score   support

           0       0.52      0.94      0.67      3273
           1       0.92      0.45      0.60      5199

    accuracy                           0.64      8472
   macro avg       0.72      0.69      0.63      8472
weighted avg       0.77      0.64      0.63      8472



In [8]:
# for undersampled class data
classifieru = GaussianNB()
classifieru.fit(X_u_train, y_u_train)
yuPredNB = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredNB))
print('\n')
print(classification_report(y_u_test,yuPredNB))

Undersampled classes
[[1158   99]
 [ 573  655]]


              precision    recall  f1-score   support

           0       0.67      0.92      0.78      1257
           1       0.87      0.53      0.66      1228

    accuracy                           0.73      2485
   macro avg       0.77      0.73      0.72      2485
weighted avg       0.77      0.73      0.72      2485



In [9]:
# for oversampled class data
classifier = GaussianNB()
classifier.fit(X_o_train, y_o_train)
yoPredNB = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredNB))
print('\n')
print(classification_report(y_o_test,yoPredNB))

Oversampled (x) classes
[[5160  134]
 [3102 2040]]


              precision    recall  f1-score   support

           0       0.62      0.97      0.76      5294
           1       0.94      0.40      0.56      5142

    accuracy                           0.69     10436
   macro avg       0.78      0.69      0.66     10436
weighted avg       0.78      0.69      0.66     10436



In [10]:
# running descision tree classifiers

In [11]:
# for unbalanced class data
classifierdf = DecisionTreeClassifier()
classifierdf.fit(X_df_train, y_df_train)
ydfPredDT = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredNB))
print('\n')
print(classification_report(y_df_test,ydfPredNB))

Unbalanced classes
[[3080  193]
 [2880 2319]]


              precision    recall  f1-score   support

           0       0.52      0.94      0.67      3273
           1       0.92      0.45      0.60      5199

    accuracy                           0.64      8472
   macro avg       0.72      0.69      0.63      8472
weighted avg       0.77      0.64      0.63      8472



In [12]:
# for undersampled class data
classifieru = GaussianNB()
classifieru.fit(X_u_train, y_u_train)
yuPredDT = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredNB))
print('\n')
print(classification_report(y_u_test,yuPredNB))

Undersampled classes
[[1158   99]
 [ 573  655]]


              precision    recall  f1-score   support

           0       0.67      0.92      0.78      1257
           1       0.87      0.53      0.66      1228

    accuracy                           0.73      2485
   macro avg       0.77      0.73      0.72      2485
weighted avg       0.77      0.73      0.72      2485



In [13]:
# for oversampled class data
classifier = DecisionTreeClassifier()
classifier.fit(X_o_train, y_o_train)
yoPredDT = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredNB))
print('\n')
print(classification_report(y_o_test,yoPredNB))

Oversampled (x) classes
[[5160  134]
 [3102 2040]]


              precision    recall  f1-score   support

           0       0.62      0.97      0.76      5294
           1       0.94      0.40      0.56      5142

    accuracy                           0.69     10436
   macro avg       0.78      0.69      0.66     10436
weighted avg       0.78      0.69      0.66     10436



In [14]:
# running KNN classifiers

In [15]:
# for unbalanced class data
classifierdf = KNeighborsClassifier(n_neighbors=1)
classifierdf.fit(X_df_train, y_df_train)
ydfPredKNN = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredNB))
print('\n')
print(classification_report(y_df_test,ydfPredNB))

Unbalanced classes
[[3080  193]
 [2880 2319]]


              precision    recall  f1-score   support

           0       0.52      0.94      0.67      3273
           1       0.92      0.45      0.60      5199

    accuracy                           0.64      8472
   macro avg       0.72      0.69      0.63      8472
weighted avg       0.77      0.64      0.63      8472



In [16]:
# for undersampled class data
classifieru = KNeighborsClassifier(n_neighbors=1)
classifieru.fit(X_u_train, y_u_train)
yuPredKNN = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredNB))
print('\n')
print(classification_report(y_u_test,yuPredNB))

Undersampled classes
[[1158   99]
 [ 573  655]]


              precision    recall  f1-score   support

           0       0.67      0.92      0.78      1257
           1       0.87      0.53      0.66      1228

    accuracy                           0.73      2485
   macro avg       0.77      0.73      0.72      2485
weighted avg       0.77      0.73      0.72      2485



In [17]:
# for oversampled class data
classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(X_o_train, y_o_train)
yoPredKNN = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredNB))
print('\n')
print(classification_report(y_o_test,yoPredNB))

Oversampled (x) classes
[[5160  134]
 [3102 2040]]


              precision    recall  f1-score   support

           0       0.62      0.97      0.76      5294
           1       0.94      0.40      0.56      5142

    accuracy                           0.69     10436
   macro avg       0.78      0.69      0.66     10436
weighted avg       0.78      0.69      0.66     10436



In [18]:
# running random forest classifier

In [19]:
# for unbalanced class data
classifierdf = RandomForestClassifier(n_estimators=100)
classifierdf.fit(X_df_train, y_df_train)
ydfPredRF = classifierdf.predict(X_df_test)
print("Unbalanced classes")
print(confusion_matrix(y_df_test,ydfPredNB))
print('\n')
print(classification_report(y_df_test,ydfPredNB))

Unbalanced classes
[[3080  193]
 [2880 2319]]


              precision    recall  f1-score   support

           0       0.52      0.94      0.67      3273
           1       0.92      0.45      0.60      5199

    accuracy                           0.64      8472
   macro avg       0.72      0.69      0.63      8472
weighted avg       0.77      0.64      0.63      8472



In [20]:
# for undersampled class data
classifieru = RandomForestClassifier(n_estimators=100)
classifieru.fit(X_u_train, y_u_train)
yuPredRF = classifieru.predict(X_u_test)
print("Undersampled classes")
print(confusion_matrix(y_u_test,yuPredNB))
print('\n')
print(classification_report(y_u_test,yuPredNB))

Undersampled classes
[[1158   99]
 [ 573  655]]


              precision    recall  f1-score   support

           0       0.67      0.92      0.78      1257
           1       0.87      0.53      0.66      1228

    accuracy                           0.73      2485
   macro avg       0.77      0.73      0.72      2485
weighted avg       0.77      0.73      0.72      2485



In [21]:
# for oversampled class data
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_o_train, y_o_train)
yoPredRF = classifier.predict(X_o_test)
print("Oversampled (x) classes")
print(confusion_matrix(y_o_test,yoPredNB))
print('\n')
print(classification_report(y_o_test,yoPredNB))

Oversampled (x) classes
[[5160  134]
 [3102 2040]]


              precision    recall  f1-score   support

           0       0.62      0.97      0.76      5294
           1       0.94      0.40      0.56      5142

    accuracy                           0.69     10436
   macro avg       0.78      0.69      0.66     10436
weighted avg       0.78      0.69      0.66     10436

